In [4]:
# Cell 1: Install Dependencies and Prepare Environment

# Install all required packages quietly
!pip install -q streamlit pyngrok bcrypt pyjwt pandas transformers rouge_score matplotlib sentencepiece torch textstat rake-nltk

# Kill existing ngrok tunnels and processes
import subprocess
import os
import signal
import sys
import time
import nltk
from pyngrok import ngrok

def kill_processes_by_name(name):
    try:
        subprocess.run(["pkill", "-9", "-f", name], check=False, stderr=subprocess.DEVNULL)
    except Exception:
        pass

ngrok.kill()
kill_processes_by_name("ngrok")
kill_processes_by_name("streamlit")

# Set ngrok auth token
# *** REPLACE with your actual ngrok token ***
NGROK_AUTH_TOKEN = "34NkoiF9Xw7WRHBGBUowsphDAKC_5Jq4aktnBRrALwdKaRqDf"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("ngrok authentication token set.")

# NLTK Setup: Fixes LookupError and punkt_tab issue
nltk_data_dir = os.path.join(os.getcwd(), "nltk_data")
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path.append(nltk_data_dir)

required_nltk_resources = ['stopwords', 'punkt', 'punkt_tab']

print("Checking and downloading NLTK data...")
for resource in required_nltk_resources:
    try:
        nltk.data.find(f'tokenizers/{resource}')
    except LookupError: # This is the standard exception thrown by nltk.data.find
        print(f"Downloading '{resource}'...")
        nltk.download(resource, download_dir=nltk_data_dir, quiet=True)
    except Exception as e:
        print(f"An unexpected error occurred while checking/downloading {resource}: {e}")

print("NLTK setup complete.")

ngrok authentication token set.
Checking and downloading NLTK data...
NLTK setup complete.


In [5]:
# Cell 2: Define and Write the Streamlit App Code

%%writefile combined_app_no_chat.py
import streamlit as st
import sqlite3
import bcrypt
import jwt
import pandas as pd
from datetime import datetime, timedelta
import random
import time
import os
import numpy as np
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from transformers import pipeline
from rouge_score import rouge_scorer
import matplotlib.pyplot as plt
import textwrap
from textstat import flesch_reading_ease, flesch_kincaid_grade, smog_index, gunning_fog
from rake_nltk import Rake
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# --- NLTK Setup ---
nltk_data_dir = os.path.join(os.getcwd(), "nltk_data")
nltk.data.path.append(nltk_data_dir)

# =============================================================================
# A. Secure Auth Backend
# =============================================================================
SECRET_KEY = "your-long-and-very-secret-key-for-jwt-goes-here"
DB_NAME = 'secure_users.db'

def send_otp_email(receiver_email, otp):
    # !!! WARNING: Replace with your actual details and app password for this to work !!!
    sender_email = "likhithalakamraju@gmail.com"  # <-- Change this
    sender_password = "umcu irpc amql uzks"  # <-- Use Gmail App Password (16 chars)
    subject = "Your OTP for Password Reset"
    body = f"""
    Hello,
    Your One-Time Password (OTP) for resetting your password is: {otp}
    This OTP is valid for 10 minutes.
    Regards,
    SecureApp Team
    """
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message.attach(MIMEText(body, "plain"))

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.send_message(message)
        return True, "OTP sent successfully!"
    except Exception as e:
        return False, f"Failed to send OTP email: {str(e)}"

def hash_password(password):
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

def verify_password(plain_password, hashed_password):
    return bcrypt.checkpw(plain_password.encode('utf-8'), hashed_password)

def init_db():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS users (
            email TEXT PRIMARY KEY,
            password_hash BLOB NOT NULL,
            role TEXT NOT NULL,
            otp TEXT,
            otp_expiry TIMESTAMP
        )
    ''')
    c.execute("SELECT * FROM users WHERE email='admin@ai'")
    if c.fetchone() is None:
        admin_email = "admin@ai"
        admin_pass = "Infosys"
        hashed_pass = bcrypt.hashpw(admin_pass.encode(), bcrypt.gensalt())
        c.execute("INSERT INTO users (email, password_hash, role) VALUES (?, ?, ?)",
                  (admin_email, hashed_pass, "Admin"))
    conn.commit()
    conn.close()

def register_user(email, password, role="General User"):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE email=?", (email,))
    if c.fetchone():
        conn.close()
        return "Email already exists."
    hashed_password = hash_password(password)
    c.execute("INSERT INTO users (email, password_hash, role) VALUES (?, ?, ?)",
              (email, hashed_password, role))
    conn.commit()
    conn.close()
    return "User registered successfully! Please log in."

def generate_token(email, role):
    payload = {'exp': datetime.utcnow() + timedelta(hours=1), 'iat': datetime.utcnow(),
               'sub': email, 'role': role}
    return jwt.encode(payload, SECRET_KEY, algorithm='HS256')

def decode_token(token):
    try:
        return jwt.decode(token, SECRET_KEY, algorithms=['HS256'])
    except:
        return None

def authenticate_user(email, password):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT password_hash, role FROM users WHERE email=?", (email,))
    result = c.fetchone()
    conn.close()
    if result:
        hashed_password_from_db, role = result
        if verify_password(password, hashed_password_from_db):
            return generate_token(email, role)
    return None

def get_all_users():
    conn = sqlite3.connect(DB_NAME)
    df = pd.read_sql_query("SELECT email, role FROM users", conn)
    conn.close()
    return df

def delete_user(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("DELETE FROM users WHERE email=?", (email,))
    conn.commit()
    conn.close()

def generate_and_store_otp(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT email FROM users WHERE email=?", (email,))
    if not c.fetchone():
        conn.close()
        return None, "Email not found."

    otp = str(random.randint(100000, 999999))
    expiry = datetime.now() + timedelta(minutes=10)

    c.execute("UPDATE users SET otp=?, otp_expiry=? WHERE email=?",
              (otp, expiry.strftime('%Y-%m-%d %H:%M:%S'), email))
    conn.commit()
    conn.close()

    sent, msg = send_otp_email(email, otp)
    if sent:
        return otp, "OTP sent successfully to your email."
    else:
        return None, msg

def verify_otp_and_reset_password(email, otp, new_password):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT otp, otp_expiry FROM users WHERE email=?", (email,))
    result = c.fetchone()

    if result:
        stored_otp, otp_expiry_str = result
        if stored_otp is None:
            conn.close()
            return "No active OTP found. Please request a reset first."

        otp_expiry = datetime.strptime(otp_expiry_str, '%Y-%m-%d %H:%M:%S')

        if stored_otp == otp and datetime.now() < otp_expiry:
            new_hash = hash_password(new_password)
            c.execute("UPDATE users SET password_hash=?, otp=NULL, otp_expiry=NULL WHERE email=?",
                      (new_hash, email))
            conn.commit()
            conn.close()
            return "Password successfully reset!"
        elif datetime.now() >= otp_expiry:
            conn.close()
            return "OTP expired. Please request a new one."
        else:
            conn.close()
            return "Invalid OTP."

    conn.close()
    return "User not found or internal error."

init_db()

# =============================================================================
# B. Summarizer & Paraphraser Backend
# =============================================================================
@st.cache_resource
def load_summarizers():
    pipes = {}
    pipes["Pegasus"] = pipeline("summarization", model="google/pegasus-xsum")
    pipes["FLAN-T5"] = pipeline("summarization", model="google/flan-t5-base")
    pipes["BART"] = pipeline("summarization", model="facebook/bart-large-cnn")
    return pipes

@st.cache_resource
def load_paraphrasers():
    pipes = {}
    pipes["FLAN-T5"] = pipeline("text2text-generation", model="google/flan-t5-base")
    pipes["BART"] = pipeline("text2text-generation", model="facebook/bart-large-cnn")
    return pipes

def compute_rouge(candidate: str, reference: str):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    try:
        scores = scorer.score(reference, candidate)
    except Exception:
        return {}
    out = {}
    for k, v in scores.items():
        out[k.upper()] = {"P": round(v.precision,4), "R": round(v.recall,4), "F1": round(v.fmeasure,4)}
    return out

def paraphrase_instruction(style: str, complexity: str, text: str):
    prefix = ""
    if complexity == "Beginner":
        prefix += "Simplify and use plain vocabulary. "
    elif complexity == "Advanced":
        prefix += "Use richer vocabulary, more complex structures. "
    if style == "Simplification":
        prefix += "Simplify the following text: "
    elif style == "Formal":
        prefix += "Rewrite the following text in a formal tone: "
    elif style == "Creative":
        prefix += "Paraphrase creatively with fresh expressions: "
    elif style == "Academic":
        prefix += "Rewrite in an academic tone: "
    return prefix + text

# =============================================================================
# C. Text Analysis Dashboard Backend (Mocking content generation for speed)
# =============================================================================
def calculate_readability(text):
    return (flesch_reading_ease(text), flesch_kincaid_grade(text), smog_index(text), gunning_fog(text))

def get_complexity_category(fk_score):
    if fk_score >= 60: return "Beginner", "#28a745"
    elif fk_score >= 30: return "Intermediate", "#ffc107"
    else: return "Advanced", "#dc3545"

def calculate_text_metrics(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    word_count = len(words)
    sentence_count = len(sentences)
    avg_sentence_len = word_count / sentence_count if sentence_count else 0
    # Hard words: 3+ vowels
    hard_words = [w for w in words if len([c for c in w if c.lower() in "aeiou"]) >= 3]
    num_hard_words = len(hard_words)
    return {"Word Count": word_count, "Sentence Count": sentence_count, "Avg Sentence Length": avg_sentence_len, "Hard Words (3+ vowels)": num_hard_words}

def generate_summary_mock(text, length, fmt="Paragraph"):
    if length=="Short": summary="🌟 [MOCK] Short summary."
    elif length=="Medium": summary="📚 [MOCK] Medium summary."
    else: summary="🧐 [MOCK] Long summary."
    if fmt=="Bullet Points": return "\n- " + "\n- ".join(summary.split(". "))
    return summary

def paraphrase_text_mock(text, complexity):
    if complexity=="Easy": return "💡 [MOCK] Easy paraphrase."
    elif complexity=="Medium": return "📝 [MOCK] Medium paraphrase."
    else: return "🎓 [MOCK] Hard paraphrase."

def extract_keywords(text, top_n=10):
    r = Rake()
    r.extract_keywords_from_text(text)
    return r.get_ranked_phrases()[:top_n]


# =============================================================================
# D. UI Functions
# =============================================================================

def admin_dashboard():
    st.title("👑 Admin Dashboard")
    tab1, tab2 = st.tabs(["User Management", "App Analytics (Placeholder)"])

    with tab1:
        st.subheader("User Database")
        users_df = get_all_users()
        st.dataframe(users_df, use_container_width=True)

        col1, col2 = st.columns(2)
        with col1:
            with st.form("add_user_form", clear_on_submit=True):
                st.subheader("➕ Add New User")
                new_email = st.text_input("New User Email")
                new_password = st.text_input("New User Password", type="password")
                new_role = st.selectbox("Assign Role", ["General User", "Admin"])
                if st.form_submit_button("Add User"):
                    if new_email and new_password:
                        message = register_user(new_email, new_password, new_role)
                        st.success(message)
                        st.rerun()
                    else:
                        st.error("Please fill in all fields.")
        with col2:
            with st.form("delete_user_form", clear_on_submit=True):
                st.subheader("⚠️ Delete User")
                user_to_delete = st.selectbox("Select User to Delete", options=users_df['email'].tolist())
                if st.form_submit_button("Delete User", type="primary"):
                    delete_user(user_to_delete)
                    st.warning(f"User '{user_to_delete}' deleted.")
                    st.rerun()

    with tab2:
        st.subheader("Application Analytics")
        st.metric("Total Users", len(get_all_users()))
        st.metric("Daily Active Users (Mock)", "123")
        chart_data = pd.DataFrame(np.random.randn(20, 3), columns=['A', 'B', 'C'])
        st.line_chart(chart_data)

def forgot_password_ui(main_content):
    with main_content:
        st.markdown("<h1>➡ Reset Password</h1>", unsafe_allow_html=True)
        st.markdown('<div class="forgot-password-container">', unsafe_allow_html=True)

        if 'reset_email' not in st.session_state: st.session_state.reset_email = None
        if 'show_otp_form' not in st.session_state: st.session_state.show_otp_form = False

        if not st.session_state.show_otp_form:
            with st.form("request_otp_form", clear_on_submit=False):
                st.subheader("Step 1: Enter your email")
                email_for_otp = st.text_input("Registered Email")
                if st.form_submit_button("Request Password Reset"):
                    if email_for_otp:
                        with st.spinner("Processing request..."):
                            email_for_otp = email_for_otp.lower()
                            otp, message = generate_and_store_otp(email_for_otp)
                        if otp:
                            st.session_state.reset_email = email_for_otp
                            st.session_state.show_otp_form = True
                            st.success(f"OTP sent to {email_for_otp}. Proceed to Step 2.")
                            st.rerun()
                        else:
                            st.error(message)
                    else:
                        st.error("Please enter your email.")

        if st.session_state.show_otp_form:
            with st.form("reset_password_form", clear_on_submit=True):
                st.subheader("Step 2: Enter OTP and New Password")
                st.info(f"Using email: **{st.session_state.reset_email}**.")
                otp_input = st.text_input("Enter OTP (6 digits)")
                new_password = st.text_input("New Password", type="password")
                confirm_password = st.text_input("Confirm New Password", type="password")
                if st.form_submit_button("Reset Password"):
                    if new_password and confirm_password and otp_input and st.session_state.reset_email:
                        if new_password == confirm_password:
                            with st.spinner("Verifying and resetting..."):
                                message = verify_otp_and_reset_password(st.session_state.reset_email, otp_input, new_password)
                            if "successfully reset" in message:
                                st.success(message + " Redirecting to login...")
                                st.session_state.show_otp_form = False
                                st.session_state.reset_email = None
                                time.sleep(1)
                                st.session_state.auth_view = "login"
                                st.rerun()
                            else:
                                st.error(message)
                        else:
                            st.error("New passwords do not match.")
                    else:
                        st.error("Please fill in all fields.")

            col_l, col_r = st.columns(2)
            with col_l:
                if st.button("Request New OTP", type="secondary"):
                    st.session_state.show_otp_form = False
                    st.rerun()
            with col_r:
                if st.button("Go back to Login", type="secondary"):
                    st.session_state.show_otp_form = False
                    st.session_state.auth_view = "login"
                    st.rerun()

        st.markdown('</div>', unsafe_allow_html=True)

# --- UI for B. Summarizer & Paraphraser ---
def summarizer_paraphraser_ui(summarizers, paraphrasers):
    st.markdown('''<div class="header"><h1>Summarize and Paraphraser Engine</h1></div>''', unsafe_allow_html=True)
    left, right = st.columns(2, gap="large")

    # SUMMARIZER
    with left:
        st.markdown('''<div class="card">''', unsafe_allow_html=True)
        st.subheader("🧠 Summarizer")
        st.markdown('''<div class="muted">Choose model and summary length — view original vs generated summary side-by-side.</div>''', unsafe_allow_html=True)

        s_text = st.text_area("Input text to summarize", height=220, key="s_text")
        s_model = st.selectbox("Summarizer model", options=list(summarizers.keys()), index=1)
        s_len = st.radio("Summary length", ["Short", "Medium", "Long"], index=1, horizontal=True)

        if st.button("Generate Summary"):
            if not s_text.strip():
                st.warning("Please paste text to summarize.")
            else:
                pipe = summarizers[s_model]
                min_l, max_l = {"Short": (20,60), "Medium": (60,140), "Long": (120,300)}[s_len]
                with st.spinner("Generating summary..."):
                    try:
                        res = pipe(s_text, min_length=min_l, max_length=max_l, do_sample=False)
                        summary = res[0]["summary_text"] if isinstance(res, list) else str(res)
                    except Exception as e:
                        summary = f"Error: {e}"

                c1, c2 = st.columns(2)
                with c1:
                    st.markdown("**Original**")
                    st.text_area("Original text", s_text, height=200, key="s_orig_disp")
                with c2:
                    st.markdown("**Generated Summary**")
                    st.text_area("Summary", summary, height=200, key="s_sum_disp")

                rouge = compute_rouge(summary, s_text)
                if rouge:
                    st.markdown("**ROUGE (summary vs original)**")
                    for metric, vals in rouge.items():
                        st.write(f"- **{metric}** — P: {vals['P']}  R: {vals['R']}  F1: {vals['F1']}")

                orig_w, sum_w = len(s_text.split()), len(summary.split())
                comp = round(sum_w / orig_w, 3) if orig_w>0 else 0
                st.markdown(f"**Original words:** {orig_w} — **Summary words:** {sum_w} — **Compression:** {comp}")

                st.session_state.summary_history.append({"original": s_text, "summary": summary})

        st.markdown("---")
        st.markdown("### 🕓 Recent Summaries")
        if st.session_state.summary_history:
            for i, rec in enumerate(reversed(st.session_state.summary_history[-6:])):
                st.markdown(f"<div class='history-item'><b>{i+1}.</b> {textwrap.shorten(rec['original'], width=140)}<br><i>Summary:</i> {textwrap.shorten(rec['summary'], width=180)}</div>", unsafe_allow_html=True)
        else:
            st.info("No summaries yet — generate one!")

        st.markdown('''</div>''', unsafe_allow_html=True)


    # PARAPHRASER
    with right:
        st.markdown('''<div class="card">''', unsafe_allow_html=True)
        st.subheader("🪄 Paraphraser")
        st.markdown('''<div class="muted">Select model, complexity, and style. Compare results with a readability line graph.</div>''', unsafe_allow_html=True)

        p_text = st.text_area("Input text to paraphrase", height=220, key="p_text")
        p_model = st.selectbox("Paraphraser model", options=list(paraphrasers.keys()), index=0)
        complexity = st.selectbox("Complexity level", options=["Beginner","Intermediate","Advanced"], index=1)
        p_style = st.selectbox("Paraphrasing style", options=["Simplification","Formal","Creative","Academic"], index=0)

        if st.button("Generate Paraphrase", key="p_generate_btn"):
            if not p_text.strip():
                st.warning("Please enter text to paraphrase.")
            else:
                pipe = paraphrasers[p_model]
                prompt = paraphrase_instruction(p_style, complexity, p_text)
                with st.spinner("Generating paraphrase..."):
                    try:
                        out = pipe(prompt, max_length=256, do_sample=True, top_p=0.92, temperature=0.95)
                        paraphrased = out[0].get("generated_text") if isinstance(out, list) else str(out)
                    except Exception as e:
                        paraphrased = f"Error generating paraphrase: {e}"

                # side-by-side comparison
                pc1, pc2 = st.columns(2)
                with pc1:
                    st.markdown("**Original**")
                    st.text_area("Original text", p_text, height=180, key="p_orig_disp")
                with pc2:
                    st.markdown("**Paraphrased**")
                    st.text_area("Paraphrased text", paraphrased, height=180, key="p_par_disp")

                # Line Graph (Readability Approximation)
                st.markdown("---")
                st.subheader("Detail Level")
                fig, ax = plt.subplots(figsize=(8, 4))
                ax.plot(["Original", "Intermediate", "Beginner"],
                        [1, 0.7, 0.4], marker="o", color="#0d6efd")
                ax.plot("Original", 1, marker="o", markersize=10, color='blue', label="Original")
                # Dynamic complexity point approximation
                complexity_y = {"Beginner": 0.4, "Intermediate": 0.7, "Advanced": 1.0}[complexity]
                ax.plot(complexity, complexity_y, marker="o", markersize=10, color='green', label="Paraphrased")
                ax.set_ylim(0, 1.2)
                ax.set_ylabel("Detail Level (relative)")
                ax.grid(axis='y', linestyle='--')
                ax.set_title("Readability Comparison")
                st.pyplot(fig)

                st.session_state.paraphrase_history.append({"original": p_text, "paraphrased": paraphrased})

        st.markdown("---")
        st.markdown("### 🕓 Recent Paraphrases")
        if st.session_state.paraphrase_history:
            for i, rec in enumerate(reversed(st.session_state.paraphrase_history[-6:])):
                st.markdown(f"<div class='history-item'><b>{i+1}.</b> {textwrap.shorten(rec['original'], width=140)}<br><i>Paraphrased:</i> {textwrap.shorten(rec['paraphrased'], width=180)}</div>", unsafe_allow_html=True)
        else:
            st.info("No paraphrases yet — generate one!")

        st.markdown('''</div>''', unsafe_allow_html=True)

# --- UI for C. Text Analysis Dashboard ---
def text_analysis_dashboard_ui():
    st.title("📊 Text Analysis & Summarization Dashboard")
    st.subheader("Your AI Toolkit for Content Refinement")

    st.markdown("---")
    st.markdown("### 📄 Upload Document or Paste Text")
    uploaded_file = st.file_uploader("Upload a text file (TXT only)", type=["txt"])
    text_input = ""
    if uploaded_file: text_input = uploaded_file.read().decode("utf-8")
    manual_text = st.text_area("Or paste your text here:", height=250)
    if manual_text: text_input = manual_text

    # --- Sidebar: Content Refinement Toolkit ---
    st.sidebar.subheader("🧠 Content Refinement Toolkit")
    summary_type = st.sidebar.selectbox("Summary Length", ["Short","Medium","Long"], index=1)
    summary_format = st.sidebar.selectbox("Summary Format", ["Paragraph","Bullet Points"], index=0)
    complexity_level = st.sidebar.selectbox("Paraphrase Complexity", ["Easy","Medium","Hard"], index=1)
    generate_btn = st.sidebar.button("Generate Refined Content")

    if text_input:
        fk_score, fk_grade, smog_score, gunning_score = calculate_readability(text_input)
        metrics = calculate_text_metrics(text_input)

        st.markdown("---")
        st.subheader("💡 Real-Time Readability Analysis")
        col1, col2, col3, col4 = st.columns(4)
        def render_scorecard(col, title, score, color):
            with col:
                st.markdown(f"<div style='padding:10px;border-radius:10px;text-align:center;background:#f8f9fa;border:1px solid #e9ecef;'><div style='font-size:2em;color:{color}'>{score:.1f}</div><div>{title}</div></div>", unsafe_allow_html=True)
        render_scorecard(col1, "Flesch-Kincaid Ease", fk_score, "#28a745" if fk_score >=60 else "#ffc107" if fk_score >=30 else "#dc3545")
        render_scorecard(col2, "Gunning Fog Score", gunning_score, "#dc3545" if gunning_score>=12 else "#ffc107" if gunning_score>=8 else "#28a745")
        render_scorecard(col3, "SMOG Index", smog_score, "#dc3545" if smog_score>=10 else "#ffc107" if smog_score>=7 else "#28a745")
        render_scorecard(col4, "Word Count", metrics["Word Count"], "#007bff")
        st.markdown(f"*Flesch-Kincaid Grade Level:* *{fk_grade:.1f}*")

        st.markdown("---")
        st.subheader("📊 Readability Scores Overview")
        scores = {
            "Flesch-Kincaid Ease": fk_score,
            "Gunning Fog Score": gunning_score,
            "SMOG Index": smog_score
        }
        colors = ["#28a745", "#ffc107", "#dc3545"]
        fig_scores, ax_scores = plt.subplots(figsize=(6,2))
        bars = ax_scores.barh(list(scores.keys()), list(scores.values()), color=colors)
        ax_scores.set_xlim(0, max(scores.values())*1.2)
        ax_scores.set_xlabel("Score Value")
        for bar in bars:
            width = bar.get_width()
            ax_scores.text(width + 0.2, bar.get_y() + bar.get_height()/2, f'{width:.1f}', va='center')
        st.pyplot(fig_scores)

        st.markdown("---")
        st.subheader("Level-Based Complexity Indicator")
        category, _ = get_complexity_category(fk_score)
        fig, ax = plt.subplots(figsize=(6,2))
        levels = ["Beginner (60+)", "Intermediate (30-59)", "Advanced (0-29)"]
        colors = ["#28a745", "#ffc107", "#dc3545"]
        data = [100 if level.startswith(category) else 0 for level in levels]
        ax.barh(levels, data, color=colors)
        ax.set_xlim(0,100)
        ax.set_xticks([])
        st.pyplot(fig)

        st.markdown("---")
        st.subheader("📑 Detailed Text Metrics")
        st.table(metrics)

        st.markdown("---")
        st.subheader("🔑 Key Phrases")
        keywords = extract_keywords(text_input)
        for k in keywords:
            st.markdown(f"- {k}")

        if generate_btn:
            st.info("Generating content...")
            summary = generate_summary_mock(text_input, summary_type, summary_format)
            paraphrase = paraphrase_text_mock(text_input, complexity_level)

            output_col1, output_col2 = st.columns(2)
            with output_col1:
                st.markdown("*Original Text*")
                st.info(text_input[:500]+("..." if len(text_input)>500 else ""))
                st.markdown(f"*Generated Summary ({summary_type},{summary_format})*")
                st.success(summary)
            with output_col2:
                st.markdown(f"*Paraphrased Text ({complexity_level} Complexity)*")
                st.warning(paraphrase)

            # Word Count Comparison Graph
            original_len = len(text_input.split())
            summary_len = len(summary.split())
            paraphrase_len = len(paraphrase.split())
            fig2, ax2 = plt.subplots(figsize=(6,4))
            categories = ["Original", f"Summary ({summary_type})", f"Paraphrase ({complexity_level})"]
            values = [original_len, summary_len, paraphrase_len]
            colors = ["#007bff","#28a745","#ffc107"]
            bars = ax2.bar(categories, values, color=colors)
            ax2.set_ylabel("Word Count")
            ax2.set_title("📊 Content Length Comparison")
            ax2.bar_label(bars, labels=[str(v) for v in values], padding=3)
            st.pyplot(fig2)

    else:
        st.info("Please upload or paste text to begin the analysis.")


# =============================================================================
# E. MAIN ROUTER LOGIC
# =============================================================================

# --- PAGE CONFIGURATION & CUSTOM CSS ---
st.set_page_config(
    page_title="AI NLP Dashboard",
    layout="wide",
    initial_sidebar_state="collapsed"
)

# Custom CSS for Auth view (FIXED with triple single quotes)
custom_css_auth = '''
<style>
    .main .block-container {
        max-width: 550px; /* Constrains Auth/Login view */
        padding-top: 2rem;
    }
    h1 {
        background-color: #0d6efd;
        color: white;
        padding: 1rem;
        border-radius: 10px 10px 0 0;
        text-align: left;
        font-size: 24px;
        margin-bottom: 0 !important;
    }
    div[data-testid="stTabs"], div.forgot-password-container {
        border: 1px solid #E0E0E0;
        border-radius: 0 0 10px 10px;
        padding: 1.5rem;
        box-shadow: 0 4px 8px rgba(0,0,0,0.05);
        background-color: white;
    }
    .stButton>button {
        width: 100%;
        background-color: #0d6efd;
        color: white;
        border-radius: 6px;
    }
    .stButton>button:hover { background-color: #0b5ed7; }
</style>
'''

# Custom CSS for Summarizer/Paraphraser view (FIXED with triple single quotes)
custom_css_sum = '''
<style>
[data-testid="stAppViewContainer"] {
  background: linear-gradient(180deg, #f3f4f6 0%, #e6f2fb 60%);
  color: #0f172a;
}
.header {
  background: linear-gradient(90deg, #e6eef9 0%, #dbeffd 50%, #d1f1ff 100%);
  border-radius: 12px;
  padding: 14px 20px;
  box-shadow: 0 4px 14px rgba(2,6,23,0.06);
  margin-bottom: 18px;
}
.header h1 { margin: 0; font-size: 22px; color: #03396c; font-weight: 700; }
.card {
  background: white;
  border-radius: 12px;
  padding: 18px;
  box-shadow: 0 6px 20px rgba(2,6,23,0.06);
  margin-bottom: 18px;
}
.history-item { background:#fbfdff; padding:8px; border-radius:8px; margin-bottom:8px; }
</style>
'''
# Dark theme CSS (FIXED with triple single quotes)
dark_theme_css = '''
<style>
    .main { background-color: #0E1117; color: #FAFAFA; }
    div[data-testid="stTabs"], div.forgot-password-container { background-color: #161B22; border: 1px solid #30363D; }
    h1, h2, h3, h4, h5, h6, .stSidebar h2 { color: #C9D1D9; }
</style>
'''

# Apply default styling
st.markdown(custom_css_auth, unsafe_allow_html=True)

# --- Session State Initialization ---
if 'token' not in st.session_state: st.session_state.token = None
if 'auth_view' not in st.session_state: st.session_state.auth_view = "login"
if 'app_view' not in st.session_state: st.session_state.app_view = "summarizer"

if "summary_history" not in st.session_state:
    st.session_state.summary_history = []
if "paraphrase_history" not in st.session_state:
    st.session_state.paraphrase_history = []

if 'theme' not in st.session_state:
    st.session_state.theme = "Light"


payload = decode_token(st.session_state.token)

if payload is None:
    # --- UNAUTHENTICATED VIEW (Login/Register/Forgot Password) ---
    st.session_state.token = None
    st.markdown("<style>div[data-testid='stHorizontalBlock'] { text-align: center; }</style>", unsafe_allow_html=True)
    _, main_content, _ = st.columns([1, 1.5, 1])

    if st.session_state.auth_view == "forgot_password":
        forgot_password_ui(main_content)
    else:
        with main_content:
            st.markdown("<h1>➡ User Authentication</h1>", unsafe_allow_html=True)
            login_tab, register_tab = st.tabs(["Login", "Register"])
            with login_tab:
                with st.form("login_form"):
                    email = st.text_input("Email", placeholder="Enter your Email").lower()
                    password = st.text_input("Password", type="password", placeholder="*****")
                    if st.form_submit_button("Sign In"):
                        token = authenticate_user(email, password)
                        if token:
                            st.session_state.token = token
                            st.rerun()
                        else:
                            st.error("Invalid email or password.")
                if st.button("Forgot Password?", key="forgot_pw_btn"):
                    st.session_state.auth_view = "forgot_password"
                    st.session_state.show_otp_form = False
                    st.rerun()

            with register_tab:
                with st.form("register_form"):
                    new_email = st.text_input("Email", key="reg_email").lower()
                    new_password = st.text_input("New Password", type="password", key="reg_pass")
                    confirm_password = st.text_input("Confirm New Password", type="password", key="reg_confirm")
                    role = st.selectbox("Role", ["General User", "Admin"])
                    if st.form_submit_button("Register"):
                        if new_password == confirm_password:
                            message = register_user(new_email, new_password, role)
                            if "successfully" in message:
                                st.success(message)
                            else:
                                st.warning(message)
                        else:
                            st.error("Passwords do not match.")
else:
    # --- AUTHENTICATED VIEW ---
    role = payload['role']
    st.session_state.auth_view = "login"

    # Sidebar for authenticated user
    with st.sidebar:
        st.success(f"Logged in as: {payload['sub']}")
        st.write(f"Your role is: **{role}**")
        st.markdown("---")
        st.subheader("🌐 Select Application")
        if role.lower() == "admin":
             # Admin can only access the Admin Dashboard
             st.session_state.app_view = "admin_dashboard"
        else:
            st.session_state.app_view = st.radio(
                "Select App",
                ["Summarizer/Paraphraser", "Text Analysis Dashboard"],
                key="app_view_radio",
                format_func=lambda x: x.split('/')[0]
            )

        st.markdown("---")
        st.subheader("⚙️ Settings")
        st.session_state.theme = st.radio("Theme", ["Light", "Dark"], key="theme_radio")
        if st.button("Logout"):
            st.session_state.token = None
            st.rerun()


    # Apply theme
    if st.session_state.theme == "Dark":
        st.markdown(dark_theme_css, unsafe_allow_html=True)
    elif st.session_state.app_view == "Summarizer/Paraphraser":
        st.markdown(custom_css_sum, unsafe_allow_html=True)


    # Run selected app
    if st.session_state.app_view == "admin_dashboard" and role.lower() == "admin":
        admin_dashboard()
    elif st.session_state.app_view == "Summarizer/Paraphraser":
        # Load models on demand for this app
        with st.spinner("Loading NLP models for Summarization & Paraphrasing... (This may take a minute on first run)"):
            summarizers = load_summarizers()
            paraphrasers = load_paraphrasers()
        summarizer_paraphraser_ui(summarizers, paraphrasers)
    elif st.session_state.app_view == "Text Analysis Dashboard":
        text_analysis_dashboard_ui()
    else:
        st.info("Select an application from the sidebar to begin.")

Writing combined_app_no_chat.py


In [6]:
# Cell 3: Execute the Streamlit App

import threading
import time
import os
from pyngrok import ngrok

# The file name is defined in Cell 2
file_name = "combined_app_no_chat.py"

def run_streamlit():
    # Start Streamlit using the combined file
    os.system(f"streamlit run {file_name} --server.port 8501 --server.headless true")

threading.Thread(target=run_streamlit, daemon=True).start()

# Wait enough for Streamlit, database, and all initial models to initialize.
print("\n⏳ Waiting for the combined app (with multiple NLP models) to start...")
time.sleep(60)

# Connect ngrok to the Streamlit port
try:
    public_url = ngrok.connect(8501)
    print(f"🌍 Your Combined Streamlit app is live at: {public_url}")
except Exception as e:
    print(f"❌ Failed to start ngrok tunnel: {e}")
    print("Please ensure the ngrok token is correct and the cell was executed without interruption.")


⏳ Waiting for the combined app (with multiple NLP models) to start...
🌍 Your Combined Streamlit app is live at: NgrokTunnel: "https://diversely-pseudopregnant-bula.ngrok-free.dev" -> "http://localhost:8501"
